<a href="https://colab.research.google.com/github/hrshwrdhn/ai_cheat/blob/main/assignment_cifar_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## go to runtime>> change runtime type >> select GPU   if you are using google colab

In [ ]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
%matplotlib inline

In [ ]:
dataset = CIFAR10(root='data/', download=True, transform=ToTensor())
test_dataset = CIFAR10(root='data/', train=False, transform=ToTensor())

In [ ]:
dataset

In [ ]:
dataset[0]

 ## Q1 . How many images does the training dataset contain

In [ ]:
# write here

ans = 50000

## Q2: How many images does the test dataset contain?


In [ ]:
# write here

## Q3: How many output classes does the dataset contain? Can you list them?

In [ ]:
# write here

ans = (10,
 ['airplane',
  'automobile',
  'bird',
  'cat',
  'deer',
  'dog',
  'frog',
  'horse',
  'ship',
  'truck'])

hint: use `dataset.classes`


## Q4:  What is the shape of an image tensor from the dataset?

In [ ]:
# write here

ans: torch.Size([3, 32, 32])

## note
 this dataset consists of 3-channel color images (RGB). Let us look at a sample image from the dataset. matplotlib expects channels to be the last dimension of the image tensors (whereas in PyTorch they are the first dimension), so we'll the .permute tensor method to shift channels to the last dimension. Let's also print the label for the image.

In [ ]:
img, label = dataset[3]
plt.imshow(img.permute((1, 2, 0)))
print('Label (numeric):', label)
print('Label (textual):', classes[label])

## Q5: Can you determine the number of images belonging to each class?

Hint: Loop through the dataset.


In [ ]:
# write here

answer
1. Number of images belonging to airplane is 5000
2. Number of images belonging to automobile is 5000
3. Number of images belonging to bird is 5000
4. Number of images belonging to cat is 5000
5. Number of images belonging to deer is 5000
6. Number of images belonging to dog is 5000
7. Number of images belonging to frog is 5000
8. Number of images belonging to horse is 5000
9. Number of images belonging to ship is 5000
10. Number of images belonging to truck is 5000

# Preparing the data for training

 use a validation set with 5000 images (10% of the dataset). To ensure we get the same validation set each time, we'll set PyTorch's random number generator to a seed value of 43.

In [ ]:
torch.manual_seed(43)
val_size = 5000
train_size = len(dataset) - val_size

## Q6: use the random_split method to create the training & validation sets

create train_ds, val_ds using function `random_split`

In [ ]:
# write here

In [ ]:
# run this line to verify 
len(train_ds), len(val_ds)

ans: (45000, 5000)

In [ ]:
batch_size=128

# Q7:   create data loaders to load the data in batches

In [ ]:
train_loader =  # write here
val_loader = # write here
test_loader = # write here

Let's visualize a batch of data using the make_grid helper function from Torchvision.

In [ ]:
for images, _ in train_loader:
    print('images.shape:', images.shape)
    plt.figure(figsize=(16,8))
    plt.axis('off')
    plt.imshow(make_grid(images, nrow=16).permute((1, 2, 0)))
    break

# Base Model class & Training on GPU

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))

## Training code function

In [ ]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result)
        history.append(result)
    return history

, let's also define some utilities for moving out data & labels to the GPU, if one is available.

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

In [ ]:
device = get_default_device()
device


In [ ]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
def plot_losses(history):
    losses = [x['val_loss'] for x in history]
    plt.plot(losses, '-x')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.title('Loss vs. No. of epochs');

In [ ]:
def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs');

In [ ]:
# moving dataloader to gpu
train_loader = DeviceDataLoader(train_loader, device)
val_loader = DeviceDataLoader(val_loader, device)
test_loader = DeviceDataLoader(test_loader, device)

Training the model

We will make several attempts at training the model. Each time, try a different architecture and a different set of learning rates. Here are some ideas to try:

1.    Increase or decrease the number of hidden layers
2.    Increase of decrease the size of each hidden layer
3.    Try different activation functions
4.    Try training for different number of epochs
5.    Try different learning rates in every epoch


In [ ]:
input_size = 3*32*32
output_size = 10

## Q8: create a model 
## ImageClassificationBase class to complete the model definition.
hint :  Define the __init__ and forward methods.
two hidden layer of size 256 and 64

In [ ]:
class CIFAR10Model(ImageClassificationBase):
    def __init__(self, in_size, out_size):
        super().__init__()
        self.linear1 = # write here)
        self.linear2 = # write here
        self.linear3 = # write here
        
        
    def forward(self, xb):
        # Flatten images into vectors
        out = xb.view(xb.size(0), -1)
        # Apply layers & activation functions
        out = self.linear1(out)
        out = F.relu6(out)
        out = self.linear2(out)
        out = F.relu6(out)
        out = self.linear3(out)
        
        return out

In [ ]:
model = to_device(CIFAR10Model(input_size, output_size), device)

In [ ]:
model

ans: 

```
CIFAR10Model(
  (linear1): Linear(in_features=3072, out_features=256, bias=True)
  (linear2): Linear(in_features=256, out_features=64, bias=True)
  (linear3): Linear(in_features=64, out_features=10, bias=True)
# )
```

In [ ]:
history = [evaluate(model, val_loader)]
history

## Q9: Train the model using the fit function to reduce the validation loss & improve accuracy
adjusting the no. of epochs & learning rate each time based on the result of the previous training phase.

In [ ]:
history += # write here

plot history

In [ ]:
plot_losses(history)

In [ ]:
plot_accuracies(history)

# q10: evaluate the model on the test dataset report its final performance.
use evaluate function

In [ ]:
a=evaluate(model, test_loader)
a


save your model


In [ ]:
torch.save(model.state_dict(), 'cifar10-feedforward.pth')